In [1]:
import pandas as pd
import os
import datetime as dt
import numpy as np

In [2]:
airport = 'FLL'
station = 74783012849

# flights

In [3]:
root = 'data/kfll/flights/'
file_list = []
csv_data = []

In [4]:
for path, subdirs, files in os.walk(root):
    for name in files:
        if name.endswith('.csv'):
            file_list.append(os.path.join(path, name))

In [5]:
flight_cols = ['Ident', 'Registration', 'Aircraft Type', 'Origin', 'Destination', 'Schedule Block Out', 'Schedule Block In',
               'Actual Block Out', 'Actual Block In', 'Actual Departure Time', 'Actual Arrival Time', 'Estimated Arrival Time',
               'Enroute Time', 'Delay', 'Cancel', 'Diversion'
       ]

In [6]:
"""
flight_dtypes = {'Marketing_Airline_Network': object,
       'Flight_Number_Marketing_Airline': int,
       'Tail_Number': object,
       'Origin': object,
       'Dest': object,
       'CRSDepTime': int,
       'DepTime': int,
       'DepDelay': float,
       'TaxiOut': int,
       'WheelsOff': int,
       'WheelsOn': int,
       'TaxiIn': int,
       'CRSArrTime': int,
       'ArrTime': int,
       'ArrDelay': float,
        'Cancelled': int,
        'CancellationCode': object,
        'Diverted': int,
        'CarrierDelay': float,
        'WeatherDelay': float,
        'NASDelay': float,
        'SecurityDelay': float,
        'LateAircraftDelay': float}
        """

"\nflight_dtypes = {'Marketing_Airline_Network': object,\n       'Flight_Number_Marketing_Airline': int,\n       'Tail_Number': object,\n       'Origin': object,\n       'Dest': object,\n       'CRSDepTime': int,\n       'DepTime': int,\n       'DepDelay': float,\n       'TaxiOut': int,\n       'WheelsOff': int,\n       'WheelsOn': int,\n       'TaxiIn': int,\n       'CRSArrTime': int,\n       'ArrTime': int,\n       'ArrDelay': float,\n        'Cancelled': int,\n        'CancellationCode': object,\n        'Diverted': int,\n        'CarrierDelay': float,\n        'WeatherDelay': float,\n        'NASDelay': float,\n        'SecurityDelay': float,\n        'LateAircraftDelay': float}\n        "

In [7]:
print(file_list)

['data/kfll/flights/FlightAware_Aersale_KFLL_Sample_2022-11-23 (2).csv']


In [8]:
for file in sorted(file_list):
    csv_file = pd.read_csv(file, usecols=lambda c: c in flight_cols, parse_dates=['Estimated Arrival Time'], encoding_errors='ignore', index_col=None, header=0, low_memory=False)
    csv_data.append(csv_file)

In [16]:
flights = csv_file

In [10]:
#flights = flights[(flights.Dest == airport)]

In [11]:
#flights.CRSArrTime = flights.CRSArrTime.astype(str).str.zfill(4)

In [12]:
#flights.CRSArrTime = pd.to_datetime(flights.CRSArrTime,format= '%H%M' ).dt.time

In [17]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ident                   534 non-null    object 
 1   Registration            534 non-null    object 
 2   Aircraft Type           534 non-null    object 
 3   Origin                  534 non-null    object 
 4   Destination             534 non-null    object 
 5   Schedule Block Out      531 non-null    object 
 6   Actual Block Out        511 non-null    object 
 7   Actual Block In         480 non-null    object 
 8   Actual Departure Time   527 non-null    object 
 9   Actual Arrival Time     0 non-null      float64
 10  Estimated Arrival Time  534 non-null    object 
 11  Enroute Time            527 non-null    object 
 12  Delay                   527 non-null    object 
 13  Cancel                  7 non-null      object 
 14  Diversion               534 non-null    ob

In [18]:
print(flights["Estimated Arrival Time"])

0      11/23/22 15:46
1      11/23/22 16:33
2      11/23/22 22:18
3       11/24/22 1:13
4      11/23/22 15:54
            ...      
529    11/23/22 14:04
530     11/23/22 3:11
531     11/23/22 3:59
532    11/23/22 17:32
533    11/23/22 23:04
Name: Estimated Arrival Time, Length: 534, dtype: object


In [48]:
flights["Estimated Arrival Time"] = pd.to_datetime(flights["Estimated Arrival Time"], errors="ignore", format='%m-%d-%Y %H:%M:%S')

In [49]:
print(flights["Estimated Arrival Time"])

0      11/23/22 15:46
1      11/23/22 16:33
2      11/23/22 22:18
3       11/24/22 1:13
4      11/23/22 15:54
            ...      
529    11/23/22 14:04
530     11/23/22 3:11
531     11/23/22 3:59
532    11/23/22 17:32
533    11/23/22 23:04
Name: Estimated Arrival Time, Length: 534, dtype: object


In [25]:
flights["Estimated Arrival Time"] = flights["Estimated Arrival Time"].dt.tz_localize('US/East', ambiguous=np.array([False] * flights.shape[0]))
# all False -> every row considered DT, alternative is True to indicate DST.

AttributeError: Can only use .dt accessor with datetimelike values

In [14]:
flights['CRSArrTimeUTC'] = flights.CRSArrTime.dt.tz_convert('UTC')

In [15]:
# print(flights[['CRSArrTime', 'CRSArrTimeUTC']])

In [16]:
flights = flights.sort_values(by='CRSArrTimeUTC')

# weather

In [34]:
root = 'data/kfll/weather/'
file_list = []
csv_data = []

In [35]:
weather_cols = [
    'DATE',
    'REPORT_TYPE',
    'CALL_SIGN',
    'VIS',
    'REM']

In [36]:
for path, subdirs, files in os.walk(root):
    for name in files:
        if name.endswith('.csv'):
            file_list.append(os.path.join(path, name))

In [37]:
for file in sorted(file_list):
    csv_file = pd.read_csv(file, usecols=lambda c: c in weather_cols, parse_dates=['DATE'], encoding_errors='ignore', index_col=None, header=0, low_memory=False)
    csv_data.append(csv_file)

In [38]:
weather = pd.concat(csv_data, ignore_index=True)

In [39]:
weather = weather[(weather.REPORT_TYPE == 'FM-15') | (weather.REPORT_TYPE == 'FM-16')]

In [40]:
weather = pd.concat([weather, weather.VIS.str.split(',', expand=True)], axis=1)
weather = weather.rename(columns={0: 'VIS_METERS', 1: 'VIS_Q', 2: 'VIS_V', 3: 'VIS_QV'})
weather = weather.drop(columns={'VIS', 'VIS_Q', 'VIS_QV'})
weather.VIS_METERS = pd.to_numeric(weather.VIS_METERS)

In [41]:
weather.DATE = pd.to_datetime(weather.DATE, utc='True')

In [42]:
weather = weather[['DATE', 'VIS_METERS', 'REM']]

In [44]:
print(weather.DATE)

0      2022-01-01 00:53:00+00:00
1      2022-01-01 01:53:00+00:00
2      2022-01-01 02:53:00+00:00
3      2022-01-01 03:53:00+00:00
4      2022-01-01 04:53:00+00:00
                  ...           
9976   2022-12-12 04:53:00+00:00
9978   2022-12-12 05:53:00+00:00
9979   2022-12-12 06:53:00+00:00
9980   2022-12-12 07:53:00+00:00
9981   2022-12-12 08:53:00+00:00
Name: DATE, Length: 9626, dtype: datetime64[ns, UTC]


# merge flights & weather

In [26]:
data = pd.merge_asof(left=flights, right=weather, left_on='CRSArrTimeUTC', right_on='DATE', direction='nearest')

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274216 entries, 0 to 274215
Data columns (total 29 columns):
 #   Column                           Non-Null Count   Dtype                      
---  ------                           --------------   -----                      
 0   FlightDate                       274216 non-null  datetime64[ns]             
 1   Marketing_Airline_Network        274216 non-null  object                     
 2   Flight_Number_Marketing_Airline  274216 non-null  int64                      
 3   Tail_Number                      273016 non-null  object                     
 4   Origin                           274216 non-null  object                     
 5   Dest                             274216 non-null  object                     
 6   CRSDepTime                       274216 non-null  int64                      
 7   DepTime                          271876 non-null  float64                    
 8   DepDelay                         271866 non-null  floa

In [28]:
data.to_csv('outputs/test.csv')